# Import software libraries

In [ ]:
import sys                                                  # Read system parameters.
import numpy as np                                          # Work with multi-dimensional arrays.
import pandas as pd                                         # Manipulate and analyze data.
import matplotlib                                           # Create and format charts.
import matplotlib.pyplot as plt
import category_encoders as ce                              # Encode data.
import sklearn                                              # Train and evaluate machine learning models.
from sklearn.model_selection import train_test_split, \
                                    learning_curve, \
                                    RandomizedSearchCV, \
                                    GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, \
                            confusion_matrix, \
                            f1_score, \
                            recall_score, \
                            precision_score, \
                            RocCurveDisplay, \
                            PrecisionRecallDisplay, \
                            ConfusionMatrixDisplay
from sklearn.dummy import DummyClassifier
import xgboost                                              # Build gradient boosting models.
from xgboost import XGBClassifier
import imblearn                                             # Deal with imbalanced data.
from imblearn.over_sampling import SMOTE                    # Perform oversampling.
from collections import Counter                             # Count objects in containers.
import pickle                                               # Save Python objects as binary files.
import warnings                                             # Suppress warnings.
warnings.filterwarnings('ignore')

# Ensure results are reproducible.
np.random.seed(1)

# Summarize software libraries used.
print('Libraries used in this project:')
print('- Python {}'.format(sys.version))
print('- NumPy {}'.format(np.__version__))
print('- pandas {}'.format(pd.__version__))
print('- Matplotlib {}'.format(matplotlib.__version__))
print('- Category Encoders {}'.format(ce.__version__))
print('- scikit-learn {}'.format(sklearn.__version__))
print('- XGBoost {}'.format(xgboost.__version__))
print('- imbalanced-learn {}\n'.format(imblearn.__version__))

# Load and preview the data

In [ ]:
users_data = pd.read_pickle('/home/student/CDSP/Classification/data/users_data_final.pickle')

users_data.head(n = 5)

# Check the shape of the data

In [ ]:
users_data.shape

# Check the data types

In [ ]:
users_data.info()

# Explore the distribution of the target variable

In [ ]:
users_data.term_deposit.value_counts(normalize = True)

# Split the data into target and features

In [ ]:
target_data = users_data.term_deposit
features = users_data.drop(['user_id', 'term_deposit'], axis = 1)

# Split the data into train and test sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, 
                                                    target_data,
                                                    test_size = 0.3)

In [ ]:
print('Training-data features: ', X_train.shape)
print('Training-data target:   ', y_train.shape)

# Apply oversampling to the data

In [ ]:
print('Before oversampling: ', Counter(y_train))

In [ ]:
# Define oversampling strategy.

SMOTE = SMOTE()

# Fit and apply the transform.

X_train_SMOTE, y_train_SMOTE = SMOTE.fit_resample(X_train, y_train)
X_train_SMOTE = pd.DataFrame(X_train_SMOTE,
                             columns = X_train.columns)

print('After oversampling: ', Counter(y_train_SMOTE))

# Check the distribution of the test data

In [ ]:
# Test data should not be oversampled.

print('Test-data features: ', X_test.shape)
print('Test-data target:   ', y_test.shape)

In [ ]:
Counter(y_test)

# Normalize the data

In [ ]:
norm = MinMaxScaler().fit(X_train_SMOTE)

In [ ]:
X_train_norm = norm.transform(X_train_SMOTE)

print('Minimum: ', np.min(X_train_norm))
print('Maximum: ', np.max(X_train_norm))

# Train a logistic regression model

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train_norm, y_train_SMOTE)

# Make predictions using the logistic regression model

In [ ]:
logreg_y_pred = logreg.predict(X_test) 
print(Counter(logreg_y_pred))

In [ ]:
results = pd.concat([y_test.iloc[:5], X_test.iloc[:5]], axis = 1)
results.insert(1, 'term_deposit_pred', logreg_y_pred[:5])
results

# Obtain the logistic regression model's score

In [ ]:
accuracy_score(y_test, logreg_y_pred)

# Train a ***k***-nearest neighbor (***k***-NN) model

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train_norm, y_train_SMOTE)

# Make predictions using the ***k***-NN model

In [ ]:
knn_y_pred = knn.predict(X_test.values)
print(Counter(knn_y_pred))

In [ ]:
results['term_deposit_pred'] = knn_y_pred[:5]
results

# Obtain the ***k***-NN model's score

In [ ]:
accuracy_score(y_test, knn_y_pred)

In [ ]:
print('Number of mislabeled points out of a total %d points: %d'
       % (X_test.shape[0], (y_test != knn_y_pred).sum()))

# Train a support-vector machine (SVM) model

In [ ]:
svm = SVC()
svm.fit(X_train_norm, y_train_SMOTE)

# Make predictions using the SVM model

In [ ]:
svm_y_pred = svm.predict(X_test)
print(Counter(svm_y_pred))

In [ ]:
results['term_deposit_pred'] = svm_y_pred[:5]
results

# Obtain the SVM model's score

In [ ]:
accuracy_score(y_test, svm_y_pred)

# Train a naïve Bayes model

In [ ]:
gnb = GaussianNB()
gnb.fit(X_train_SMOTE, y_train_SMOTE)

# Make predictions using the naïve Bayes model

In [ ]:
gnb_y_pred = gnb.predict(X_test)
print(Counter(gnb_y_pred))

In [ ]:
results['term_deposit_pred'] = gnb_y_pred[:5]
results

# Obtain the naïve Bayes model's score

In [ ]:
accuracy_score(y_test, gnb_y_pred)

# Train a decision-tree model

In [ ]:
clf_tree = DecisionTreeClassifier()
clf_tree.fit(X_train_SMOTE, y_train_SMOTE)

# Make predictions using the decision-tree model

In [ ]:
clf_tree_y_pred = clf_tree.predict(X_test)
print(Counter(clf_tree_y_pred))

In [ ]:
results['term_deposit_pred'] = clf_tree_y_pred[:5]
results

# Obtain the decision-tree model's score

In [ ]:
accuracy_score(y_test, clf_tree_y_pred)

# Visualize the decision tree

In [ ]:
text_representation = tree.export_text(clf_tree)
print(text_representation)

In [ ]:
fig = plt.figure(figsize = (25, 20))
_ = tree.plot_tree(clf_tree, 
                   feature_names = list(X_train.columns),  
                   class_names = ['No term deposit', 'Term deposit'],
                   max_depth = 2,
                   filled = True)

# Train a random-forest model

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train_SMOTE, y_train_SMOTE)

# Make predictions using the random-forest model

In [ ]:
rf_y_pred = rf.predict(X_test)
print(Counter(rf_y_pred))

In [ ]:
results['term_deposit_pred'] = rf_y_pred[:5]
results

# Obtain the random-forest model's score

In [ ]:
accuracy_score(y_test, rf_y_pred)

# Train a gradient-boosting model

In [ ]:
xgb = XGBClassifier()
xgb.fit(X_train_SMOTE, y_train_SMOTE)

# Make predictions using the gradient-boosting model

In [ ]:
xgb_y_pred = xgb.predict(X_test)
print(Counter(xgb_y_pred))

In [ ]:
results['term_deposit_pred'] = xgb_y_pred[:5]
results

# Obtain the gradient-boosting model's score

In [ ]:
accuracy_score(y_test, xgb_y_pred)

# Define the parameter grid used to tune the logistic regression model

In [ ]:
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [10, 1.0, 0.1, 0.01]

param_grid = dict(solver = solvers, penalty = penalty, C = c_values)

print(param_grid)

# Perform a randomized search for optimal hyperparameters

In [ ]:
model = LogisticRegression()
random_search = RandomizedSearchCV(estimator = model,
                                   param_distributions = param_grid)
random_search.fit(X_train_norm, y_train_SMOTE)

In [ ]:
# Summarize the results of the randomized search.

print('Best accuracy score:', round(random_search.best_score_, 4))
print('Best parameters:    ', random_search.best_params_)

# Perform a grid search for optimal hyperparameters

In [ ]:
model = LogisticRegression()
grid_search = GridSearchCV(estimator = model,
                           param_grid = param_grid)
logreg_fit = grid_search.fit(X_train_norm, y_train_SMOTE)

In [ ]:
# Summarize the results of the grid search.

print('Best accuracy score:', round(grid_search.best_score_, 4))
print('Best parameters:    ', grid_search.best_params_)

# Tune the gradient-boosting model to reduce overfitting

In [ ]:
model = XGBClassifier()

param_grid = {
    'n_estimators': [10],
    'max_depth': [15, 20, 25],
    'reg_alpha': [1.1, 1.2, 1.3],
    'reg_lambda': [1.1, 1.2, 1.3]
}

In [ ]:
gs = GridSearchCV(estimator = model,
                  param_grid = param_grid, 
                  n_jobs = -1, 
                  scoring = 'accuracy',
                  verbose = 2)

fitted_model = gs.fit(X_train_SMOTE, y_train_SMOTE)

In [ ]:
print('Best accuracy score:', round(gs.best_score_, 4))
print('Best parameters:    ', gs.best_params_)

# Compare evaluation metrics for each model

In [ ]:
models = ['Logistic Regression', 'Naïve Bayes', 'SVM', 'k-NN',
          'Decision Tree', 'Random Forest', 'XGBoost', 'Dummy Classifier']

metrics = ['Accuracy', 'Precision', 'Recall', 'F1']

pred_list = ['logreg_y_pred', 'gnb_y_pred', 'svm_y_pred', 'knn_y_pred',
             'clf_tree_y_pred','rf_y_pred', 'xgb_y_pred', 'dummy_y_pred']

# Baseline algorithm.
dummy = DummyClassifier(strategy = 'stratified')
dummy.fit(X_train_SMOTE, y_train_SMOTE)
dummy_y_pred = dummy.predict(X_test)

scores = np.empty((0, 4))

for i in pred_list:
    scores = np.append(scores,
                       np.array([[accuracy_score(y_test, globals()[i]),
                                  precision_score(y_test, globals()[i]),
                                  recall_score(y_test, globals()[i]),
                                  f1_score(y_test, globals()[i])]]),
                       axis = 0)

scores = np.around(scores, 4)

scoring_df = pd.DataFrame(scores, index = models, columns = metrics)
scoring_df.sort_values(by = 'F1', ascending = False)

# Generate a confusion matrix

In [ ]:
confusion_matrix(y_test, xgb_y_pred) 

In [ ]:
ConfusionMatrixDisplay.from_estimator(xgb, 
                                      X_test, 
                                      y_test,
                                      cmap = plt.cm.Blues,
                                      normalize = 'true')

plt.show();

# Plot a ROC curve

In [ ]:
RocCurveDisplay.from_estimator(xgb, X_test, y_test)  
plt.show()

# Plot a precision–recall curve

In [ ]:
PrecisionRecallDisplay.from_estimator(xgb, X_test, y_test)

plt.show();

# Generate a feature-importance plot

In [ ]:
def feature_importance_plot(model, X_train, n):
    """Plots feature importance. Only works for ensemble learning."""
    plt.figure(figsize = (8, 5))
    feat_importances = pd.Series(model.feature_importances_,
                                 index = X_train.columns)
    feat_importances.nlargest(n).plot(kind = 'barh')
    plt.title(f'Top {n} Features')
    plt.show()

In [ ]:
feature_importance_plot(xgb, X_train_SMOTE, 15)

# Plot learning curves

In [ ]:
def plot_learning_curves(model, X_train, y_train):
    """Plots learning curves for model validation."""
    plt.figure(figsize = (5, 5))
    train_sizes, train_scores, test_scores = \
    learning_curve(model, X_train, y_train, cv = 5,
                   scoring = 'accuracy', n_jobs = -1,
                   shuffle = True,
                   train_sizes = np.linspace(0.01, 1.0, 5))
        
    # Means of training and test set scores.
    train_mean = np.mean(train_scores, axis = 1)
    test_mean = np.mean(test_scores, axis = 1)

    # Draw lines.
    plt.plot(train_sizes, train_mean, '--',
             color = '#111111', label = 'Training score')
    plt.plot(train_sizes, test_mean,
             color = '#111111', label = 'Cross-validation score')

    # Create plot.
    plt.title('Learning Curves')
    plt.xlabel('Training Set Size')
    plt.ylabel('Accuracy Score')
    plt.legend(loc = 'best')
    plt.tight_layout()

    plt.show()

In [ ]:
plot_learning_curves(xgb, X_train_SMOTE, y_train_SMOTE)

# Save the best model

In [ ]:
pickle.dump(xgb, open('xgboost_classifier.pickle', 'wb'))